# ENTREGABLE 1 - EXTRAER DATOS DE API DE SPOTIFY

Para obtener credenciales y usar la API de Spotify, necesitas registrarte como desarrollador en la plataforma de Spotify y crear una aplicación. Aquí están los pasos generales que debes seguir:

Crea una cuenta de Spotify: Si no tienes una cuenta de Spotify, crea una en el sitio web de Spotify.

Inicia sesión en el Panel de Desarrolladores de Spotify: Visita el Panel de Desarrolladores de Spotify en https://developer.spotify.com/dashboard/login.

Crea una nueva aplicación: Una vez que hayas iniciado sesión, puedes crear una nueva aplicación. Proporciona un nombre y una descripción para tu aplicación.

Configura los ajustes de la aplicación:

Especifica la descripción y los detalles de la aplicación.
Indica si tu aplicación es comercial o personal.
Proporciona la dirección de correo electrónico de contacto.
Selecciona las capacidades que tu aplicación usará (por ejemplo, acceder a datos de usuario, controlar dispositivos, etc.).
Aceptar los términos y condiciones: Asegúrate de leer y aceptar los términos y condiciones de la plataforma de desarrolladores de Spotify.

Obtén las credenciales de API:

Después de crear la aplicación, se te proporcionarán las credenciales de API, que generalmente incluyen un ID de cliente (client ID) y una clave secreta (client secret). Estas credenciales serán necesarias para autenticarte y realizar solicitudes a la API de Spotify.
Configura redireccionamientos de URI: En la configuración de la aplicación, es posible que necesites especificar los redireccionamientos de URI permitidos, que se utilizan en el flujo de autenticación. Esto depende del tipo de autenticación que utilices.

Utiliza las credenciales en tu aplicación: Ahora puedes usar las credenciales de API en tu aplicación para autenticarte y realizar solicitudes a la API de Spotify.

Es importante tener en cuenta que la API de Spotify puede requerir diferentes tipos de autenticación según tus necesidades. Puedes encontrar más información sobre cómo autenticarte y cómo realizar solicitudes específicas en la documentación oficial de la API de Spotify.

Recuerda también que es fundamental cumplir con las políticas de uso y términos de servicio de Spotify al desarrollar aplicaciones que interactúen con su plataforma.

In [82]:
!pip install spotipy
!pip install wheel
!pip install pandas
!pip install psycopg2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
#results = {
#    'tracks': {
#        'items': [
#            # Lista de pistas (cada pista es un diccionario)
#            {
#                'id': '...',
#                'name': '...',
#                'artists': [...],
#                'album': {...},
#                'duration_ms': ...,
#                'popularity': ...
#                # Otros campos específicos de la pista
#            },
#            # Más pistas...
#        ]
#    }
#}

In [84]:
# Guardo mi contraseña "client_secret" de la API de Spotify en un archivo .txt por cuestiones de seguridad y que no aparezca visible mi contraseña en el código:
with open("C:/Users/cnieto1/Desktop/Curso Data Engineering - Coderhouse/Clases/Entregable 1/client_secret_spotify.txt",'r') as f:
    pwd= f.read()

In [85]:
#Chequeo si la variable pwd tomó bien el archivo txt:
import os

file_path = "C:/Users/cnieto1/Desktop/Curso Data Engineering - Coderhouse/Clases/Entregable 1/client_secret_spotify.txt"
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        #pwd = 'cbc24622177244b7a7a7db138da28ae7'
        pwd = f.read()
else:
    print("El archivo no existe en la ruta especificada.")

In [86]:
#Ingreso mis credenciales de la API de Spotify:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import datetime
import time

client_id = 'dbe61651ee31461681339d9d1780f672'
client_secret = pwd

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret))

In [87]:
#Consultando los datos de la API de Spotify:
import pandas as pd
results = sp.search(q='year:2023', type='track', limit=50)
data = {'Id': [],'Artista': [], 'Cancion': [],'Duracion_ms': [], 'Genero': [],'Album': [], 'Album_img': [], 'Total_canciones_album': [], 'Popularidad': [], 'fecha_lanzamiento': [], 'Audio_preview': []}
for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    duration_ms = track['duration_ms']
    track_id = track['id']
    album_group = track['album']['name']
    album_img = track['album']['images'][0]['url'] #imagen de album
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date']
    preview_url = track['preview_url']
    #Quitar las comillas 
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    #Separar el género por coma
    track_genre = ', '.join(track_genre)

    data['Id'].append(id)
    data['Artista'].append(artist_name)
    data['Cancion'].append(track_name)
    data['Duracion_ms'].append(duration_ms)
    data['Album'].append(album_group)
    data['Album_img'].append(album_img)
    data['Total_canciones_album'].append(album_cont)
    data['Genero'].append(track_genre)
    data['Popularidad'].append(track_popularity)
    data['fecha_lanzamiento'].append(track_year)
    data['Audio_preview'].append(preview_url)


df = pd.DataFrame(data)
#Evitar que haya canciones duplicadas
df.drop_duplicates(subset=['Artista', 'Cancion','Album'], keep='first', inplace=True)
#Reemplazar valores nulos o vacios en el campo Género por Desconocido
df['Genero'].fillna('Desconocido', inplace=True)
df.loc[df['Genero'] == '', 'Genero'] = 'Desconocido'
#Evitar que se cargue una canción con duración 0 ms
df = df[df['Duracion_ms'] != 0]
#Verificar que la fecha se muestre en formato fecha 
df['fecha_lanzamiento'] = pd.to_datetime(df['fecha_lanzamiento'], format='%Y-%m-%d')
#display(df)
df.head()

,Id,Artista,Cancion,Duracion_ms,Genero,Album,Album_img,Total_canciones_album,Popularidad,fecha_lanzamiento,Audio_preview
0,4KULAymBBJcPRpk1yO4dOG,Zach Bryan,I Remember Everything (feat. Kacey Musgraves),227195,"classic oklahoma country, modern country pop",Zach Bryan,https://i.scdn.co/image/ab67616d0000b273e5a25e...,16,94,2023-08-25,https://p.scdn.co/mp3-preview/22b240f7ef2eff38...
1,4rXLjWdF2ZZpXCVTfWcshS,Gunna,fukumean,125040,"atl hip hop, melodic rap, rap, trap",a Gift & a Curse,https://i.scdn.co/image/ab67616d0000b273017d5e...,15,95,2023-06-16,https://p.scdn.co/mp3-preview/f237ab921af697ba...
2,2IGMVunIBsBLtEQyoI1Mu7,Doja Cat,Paint The Town Red,231750,"dance pop, pop",Paint The Town Red,https://i.scdn.co/image/ab67616d0000b2737acee9...,1,99,2023-08-04,https://p.scdn.co/mp3-preview/1d4dc1cd67ba78af...
3,2YSzYUF3jWqb9YP9VXmpjE,Drake,IDGAF (feat. Yeat),260111,"canadian hip hop, canadian pop, hip hop, pop r...",For All The Dogs,https://i.scdn.co/image/ab67616d0000b2737d3845...,23,93,2023-10-06,None
4,3vkCueOmm7xQDoJ17W1Pm3,Mitski,My Love Mine All Mine,137773,"brooklyn indie, pov: indie",The Land Is Inhospitable and So Are We,https://i.scdn.co/image/ab67616d0000b27334f21d...,11,95,2023-09-15,https://p.scdn.co/mp3-preview/2f90c9a6a2b6e385...


In [88]:
# Construyo mi Dataframe con las columnas que considero más importantes y con el orden que le indico:
#df = {'Id': [],'Artista': [], 'Cancion': [],'Duracion_ms': [],'Album': [], 'Album_img': [], 'Total_canciones_album': [], 'Popularidad': [],'Genero':[] 'fecha_lanzamiento': [], 'Insert_date':[]}
#data = pd.DataFrame()

#Agrego la columna de 'Insert_date' a mi dataframe "data":
from datetime import datetime
current_date = datetime.now().strftime('%Y-%m-%d')

df['Insert_date'] = current_date

df.head()

,Id,Artista,Cancion,Duracion_ms,Genero,Album,Album_img,Total_canciones_album,Popularidad,fecha_lanzamiento,Audio_preview,Insert_date
0,4KULAymBBJcPRpk1yO4dOG,Zach Bryan,I Remember Everything (feat. Kacey Musgraves),227195,"classic oklahoma country, modern country pop",Zach Bryan,https://i.scdn.co/image/ab67616d0000b273e5a25e...,16,94,2023-08-25,https://p.scdn.co/mp3-preview/22b240f7ef2eff38...,2023-10-25
1,4rXLjWdF2ZZpXCVTfWcshS,Gunna,fukumean,125040,"atl hip hop, melodic rap, rap, trap",a Gift & a Curse,https://i.scdn.co/image/ab67616d0000b273017d5e...,15,95,2023-06-16,https://p.scdn.co/mp3-preview/f237ab921af697ba...,2023-10-25
2,2IGMVunIBsBLtEQyoI1Mu7,Doja Cat,Paint The Town Red,231750,"dance pop, pop",Paint The Town Red,https://i.scdn.co/image/ab67616d0000b2737acee9...,1,99,2023-08-04,https://p.scdn.co/mp3-preview/1d4dc1cd67ba78af...,2023-10-25
3,2YSzYUF3jWqb9YP9VXmpjE,Drake,IDGAF (feat. Yeat),260111,"canadian hip hop, canadian pop, hip hop, pop r...",For All The Dogs,https://i.scdn.co/image/ab67616d0000b2737d3845...,23,93,2023-10-06,None,2023-10-25
4,3vkCueOmm7xQDoJ17W1Pm3,Mitski,My Love Mine All Mine,137773,"brooklyn indie, pov: indie",The Land Is Inhospitable and So Are We,https://i.scdn.co/image/ab67616d0000b27334f21d...,11,95,2023-09-15,https://p.scdn.co/mp3-preview/2f90c9a6a2b6e385...,2023-10-25


In [89]:
# Guardo mi contraseña "pwd_redshift" de mi cuenta de Redshift en un archivo .txt por cuestiones de seguridad y que no aparezca visible mi contraseña en el código:
with open("C:/Users/cnieto1/Desktop/Curso Data Engineering - Coderhouse/Clases/Entregable 1/redshift_password.txt",'r') as f:
    pwd_redshift= f.read()

In [90]:
# Creando la conexión a Redshift
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="christian_r_coderhouse"

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        #password='3b4LjN1alG',
        password=pwd_redshift,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [91]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS canciones
       (
    id VARCHAR(50) primary key  
    ,artista VARCHAR(255)   
    ,cancion VARCHAR(255)
    ,duracion_ms INTEGER
    ,genero VARCHAR(300)   
    ,album VARCHAR(100)
    ,album_img VARCHAR(300)
    ,total_canciones_album INTEGER  
    ,Popularidad INTEGER 
    ,fecha_lanzamiento date   
    ,Audio_preview NVARCHAR(300)
    ,Insert_date date
        )
    """)
    conn.commit()

In [92]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
  cur.execute("Truncate table canciones")
  count = cur.rowcount


In [93]:
#Eliminar la tabla para evitar duplicados o inconsistencias
#with conn.cursor() as cur:
#    cur.execute("Drop table canciones")
#    count = cur.rowcount

In [94]:
#consultando la tabla canciones
cur = conn.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()
results

[]

In [95]:
df.dtypes

Id                               object
Artista                          object
Cancion                          object
Duracion_ms                       int64
Genero                           object
Album                            object
Album_img                        object
Total_canciones_album             int64
Popularidad                       int64
fecha_lanzamiento        datetime64[ns]
Audio_preview                    object
Insert_date                      object
dtype: object

In [96]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO canciones (Id, Artista, Cancion, Duracion_ms, Genero, Album, Album_img, Total_canciones_album, Popularidad, fecha_lanzamiento, Audio_preview, Insert_date)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()
    

In [97]:
# Veo como quedó la tabla en Redshift luego de hacer los Insert:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()

In [98]:
# Veo cómo quedó la tabla "canciones" en Redshift:
column_names=['Id', 'Artista', 'Cancion', 'Genero', 'Album', 'Total_canciones_album', 'Popularidad', 'Fecha de Lanzamiento','Duracion_ms','Album_img','Audio_preview','Insert_date']
df = pd.DataFrame(results, columns=column_names)
df.head()

,Id,Artista,Cancion,Genero,Album,Total_canciones_album,Popularidad,Fecha de Lanzamiento,Duracion_ms,Album_img,Audio_preview,Insert_date
0,4KULAymBBJcPRpk1yO4dOG,Zach Bryan,I Remember Everything (feat. Kacey Musgraves),"classic oklahoma country, modern country pop",Zach Bryan,16,94,2023-08-25,227195,https://i.scdn.co/image/ab67616d0000b273e5a25e...,https://p.scdn.co/mp3-preview/22b240f7ef2eff38...,2023-10-25
1,4rXLjWdF2ZZpXCVTfWcshS,Gunna,fukumean,"atl hip hop, melodic rap, rap, trap",a Gift & a Curse,15,95,2023-06-16,125040,https://i.scdn.co/image/ab67616d0000b273017d5e...,https://p.scdn.co/mp3-preview/f237ab921af697ba...,2023-10-25
2,2IGMVunIBsBLtEQyoI1Mu7,Doja Cat,Paint The Town Red,"dance pop, pop",Paint The Town Red,1,99,2023-08-04,231750,https://i.scdn.co/image/ab67616d0000b2737acee9...,https://p.scdn.co/mp3-preview/1d4dc1cd67ba78af...,2023-10-25
3,2YSzYUF3jWqb9YP9VXmpjE,Drake,IDGAF (feat. Yeat),"canadian hip hop, canadian pop, hip hop, pop r...",For All The Dogs,23,93,2023-10-06,260111,https://i.scdn.co/image/ab67616d0000b2737d3845...,None,2023-10-25
4,3vkCueOmm7xQDoJ17W1Pm3,Mitski,My Love Mine All Mine,"brooklyn indie, pov: indie",The Land Is Inhospitable and So Are We,11,95,2023-09-15,137773,https://i.scdn.co/image/ab67616d0000b27334f21d...,https://p.scdn.co/mp3-preview/2f90c9a6a2b6e385...,2023-10-25


In [99]:
cur.close()
conn.close()